# Lab 1: Independent Component Analysis

### Machine Learning 2 (2019)

* The lab exercises can be done in groups of two people, or individually.
* The deadline is Tuesday September 17th, 16:59.
* Assignment should be submitted through Canvas! Make sure to include your and your teammates' names with the submission.
* Attach the .IPYNB (IPython Notebook) file containing your code and answers. Naming of the file should be "studentid1\_studentid2\_lab#", for example, the attached file should be "12345\_12346\_lab1.ipynb". Only use underscores ("\_") to connect ids, otherwise the files cannot be parsed.

Notes on implementation:

* You should write your code and answers in an IPython Notebook: http://ipython.org/notebook.html. If you have problems, please ask.
* Use __one cell__ for code and markdown answers only!
    * Put all code in the cell with the ```# YOUR CODE HERE``` comment and overwrite the ```raise NotImplementedError()``` line.
    * For theoretical questions, put your solution using LaTeX style formatting in the YOUR ANSWER HERE cell.
* Among the first lines of your notebook should be "%pylab inline". This imports all required modules, and your plots will appear inline.
* Large parts of you notebook will be graded automatically. Therefore it is important that your notebook can be run completely without errors and within a reasonable time limit. To test your notebook before submission, select Kernel -> Restart \& Run All.

### Literature
In this assignment, we will implement the Independent Component Analysis algorithm as described in chapter 34 of David MacKay's book "Information Theory, Inference, and Learning Algorithms", which is freely available here:
http://www.inference.phy.cam.ac.uk/mackay/itila/book.html

Read the ICA chapter carefuly before you continue!

### Notation

$\mathbf{X}$ is the $M \times T$ data matrix, containing $M$ measurements at $T$ time steps.

$\mathbf{S}$ is the $S \times T$ source matrix, containing $S$ source signal values at $T$ time steps. We will assume $S = M$.

$\mathbf{A}$ is the mixing matrix. We have $\mathbf{X} = \mathbf{A S}$.

$\mathbf{W}$ is the matrix we aim to learn. It is the inverse of $\mathbf{A}$, up to indeterminacies (scaling and permutation of sources).

$\phi$ is an elementwise non-linearity or activation function, typically applied to elements of $\mathbf{W X}$.

### Code
In the following assignments, you can make use of the signal generators listed below.



In [ ]:
%pylab inline
import sys
assert sys.version_info[:3] >= (3, 6, 0), "Make sure you have Python 3.6 installed"

# Signal generators
def sawtooth(x, period=0.2, amp=1.0, phase=0.):
    return (((x / period - phase - 0.5) % 1) - 0.5) * 2 * amp

def sine_wave(x, period=0.2, amp=1.0, phase=0.):
    return np.sin((x / period - phase) * 2 * np.pi) * amp

def square_wave(x, period=0.2, amp=1.0, phase=0.):
    return ((np.floor(2 * x / period - 2 * phase - 1) % 2 == 0).astype(float) - 0.5) * 2 * amp

def triangle_wave(x, period=0.2, amp=1.0, phase=0.):
    return (sawtooth(x, period, 1., phase) * square_wave(x, period, 1., phase) + 0.5) * 2 * amp

def random_nonsingular_matrix(d=2):
    """
    Generates a random nonsingular (invertible) matrix of shape d*d
    """
    epsilon = 0.1
    A = np.random.rand(d, d)
    while abs(np.linalg.det(A)) < epsilon:
        A = np.random.rand(d, d)
    return A

def plot_signals(X, title="Signals"):
    """
    Plot the signals contained in the rows of X.
    """
    figure()
    
    for i in range(X.shape[0]):
        ax = plt.subplot(X.shape[0], 1, i + 1)
        plot(X[i, :])
        ax.set_xticks([])
        ax.set_yticks([])
    plt.suptitle(title)

The following code generates some toy data to work with.

In [ ]:
# Generate data
num_sources = 5
signal_length = 500
t = linspace(0, 1, signal_length)
S = np.c_[sawtooth(t), sine_wave(t, 0.3), square_wave(t, 0.4), triangle_wave(t, 0.25), np.random.randn(t.size)].T
plot_signals(S)

### 1.1 Make mixtures (5 points)
Write a function `make_mixtures(S, A)' that takes a matrix of source signals $\mathbf{S}$ and a mixing matrix $\mathbf{A}$, and generates mixed signals $\mathbf{X}$.

In [ ]:
### 1.1 Make mixtures
def make_mixtures(S, A):
    return np.matmul(A,S)

In [ ]:
### Test your function
np.random.seed(42)
A = random_nonsingular_matrix(d=S.shape[0])
X = make_mixtures(S, A)
plot_signals(X, "Mixtures")

assert X.shape == (num_sources, signal_length), "The shape of your mixed signals is incorrect"


### 1.2 Histogram (5 points)
Write a function `plot_histograms(X)` that takes a data-matrix $\mathbf{X}$ and plots one histogram for each signal (row) in $\mathbf{X}$. You can use the `np.histogram()` (followed by `plot`) or `plt.hist()` function. 

Plot histograms of the sources and the measurements.

In [ ]:
### 1.2 Histogram
def plot_histograms(X, title="Hist"):
    
    subs = X.shape[0] if len(X.shape) == 2 else 1
    figure(figsize=(20, subs))
    
    # multiple hists
    if len(X.shape) == 2:
        for i, row in enumerate(X, start=1):
            plt.subplot(1, subs, i)
            plt.hist(row)
            
            if type(title) == list:
                plt.title(f"{title[i-1]} {i-1}")
            else:
                plt.title(f"{title} {i-1}")
    
    # one hist
    else: 
        plt.hist(X)
        plt.title(f"{title}")
        

    plt.show()

def make_title(main, additive):
    return [main + "_" + add for add in additive]

signals = ["sawtooth", "sine", "square", "triangle", "normal"]

X_title = make_title("X", signals)
plot_histograms(X, title=X_title)

S_title = make_title("S", signals)
plot_histograms(S, title=S_title)

Which of these distributions (sources or measurements) tend to look more like Gaussians? Can you think of an explanation for this phenomenon? Why is this important for ICA?

***

**ANSWER:**

***Comparing Source with Mixed distribution:*** We see that mixed distributions (measurements, $X = AS$) exhibit more Gaussian-like histograms. This is because of the Central Limit Theorem that states that when independent random variables are added, the resultant distribution tends to a normal distribution even if the original distributions were not normal. In our case, we mix 5 random distributions (weighted by $A$) and hence we observe similar behavior.

***Mixed signals:*** The *random mixed signal* looks like Gaussian distribution the most. This is because $np.random.randn$ is used to generate the signal which internally actually uses the standard normal by default to do so. The *sine* and *triangle* mixed distributions come next and this is because they have similar smooth oscillations about $0$ with very less time at the peak values and most of the time spent in transitions. *Sawtooth* is third because of similar behavior but with drastic transitions. The *square* is last as it has 2 modes in original form and switches between them drastically thus spending most of the time at these peaks.

***Source signals:*** The source signals exhibit the least Gaussian like behavior as they are all different distributions and have not been moxed yet for the Central Limit Theorem to apply.

This is not suitable for ICA as the ICA algorithm works under the assumption that the source signals ***do not*** have a Gaussian distribution. 

***

### 1.3 Implicit priors (20 points)
As explained in MacKay's book, an activation function $\phi$ used in the ICA learning algorithm corresponds to a prior distribution over sources. Specifically, $\phi(a) = \frac{d}{da} \ln p(a)$. For each of the following activation functions, *derive* the source distribution they correspond to.
$$\phi_0(a) = -\tanh(a)$$
$$\phi_1(a) = -a + \tanh(a)$$
$$\phi_2(a) = -a^3$$
$$\phi_3(a) = -\frac{6a}{a^2 + 5}$$

Give your answer without the normalizing constant, so an answer of the form $p(a) \propto \verb+[answer]+$ is ok.

**ANSWER**

***
We have,

$\phi(a)=\frac{d}{d a} \ln p(a)$ 

$\Rightarrow \int \phi(a) da=\ln p(a)$

In general, we will also use the following results for the first two part solutions:

$tanh(a) = \frac{\exp^a - \exp^{-a}}{\exp^a + \exp^{-a}}$
$   =  \frac{1 - \exp^{-2a}}{1 + \exp^{-2a}} $

Substituting $y = \exp^{-2a}$

$\Rightarrow a = \frac{-1}{2} \log y$
$\Rightarrow \frac{da}{dy} = \frac{-1}{2y}$
$\Rightarrow da = \frac{-1}{2y} dy$

Thus,

$tanh(a) = \frac{-1}{2} \int \frac{1 - y}{(1 + y)y}dy$

***

***
**1) $\ \ \  \phi_0(a)= -tanh(a)$**

$\Rightarrow \int \phi(a)da = -\int tanh(a) da$

$ =  \frac{1}{2} \int \frac{1 - y}{(1 + y)y} dy$

$ = \frac{1}{2} \int \frac{1 + y - 2y}{(1 + y)y} dy$

$ = \frac{1}{2} \int \frac{1 + y}{(1 + y)y}dy - \frac{1}{2} \int \frac{2y}{(1 + y)y}dy$

$ = \frac{1}{2} \int \frac{1}{y}dy - \int \frac{1}{(1 + y)}dy$

$ = \frac{1}{2} \ln y - \ln (1+y) + c$

$ = -a - \ln (1 + \exp^{-2a}) + c$

Thus,

$\ln p(a) = -a - \ln (1 + \exp^{-2a}) + c$

$p(a) = \exp{-a - \ln (1 + \exp^{-2a}) + c}$

$p(a) = \frac{\exp^c} {\exp^a \cdot (1 + \exp^{-2a})}$

$p(a) \propto \frac{1} {\exp^a \cdot (1 + \exp^{-2a})}$

***

***
**2) $\ \ \  \phi_1(a)= -a + tanh(a)$**

$\Rightarrow \int \phi(a)da = \int -a da + \int tanh(a) da$

Following similar steps as above, we arrive at:

$\ln p(a) = -\int a d a- (-a-\ln (1+e^{-2 a})+c)=-\frac{1}{2} a^{2}+a+\ln (1+e^{-2 a})+c$

Hence,

$p(a) \propto e^{-\frac{1}{2} a^{2}+a}+e^{-\frac{1}{2} a^{2}-a}$

***

***
**3)$\ \ \  \phi_2(a)= -a^3$**

$\Rightarrow \int \phi(a)da = \int -a^3 da$

$ = \frac{-1}{4}a^4 + c$

Thus,

$p(a) \propto \exp^{\frac {-1}{4}a^4}$

***

***
**4)$\ \ \  \phi_3(a)= - \frac{6a}{a^2 + 5}$**

$\Rightarrow \int \phi(a)da = \int-\frac{6 a}{a^{2}+5} da$

$ = -3\int \frac{2a}{a^2 + 5}da$

$= -3\ln |a^2+5|+c$

Thus,

$p(a) \propto (a^2 + 5)^{-3}$
***

In [ ]:
def phi_0(a):
    return -np.tanh(a)

def p_0(a):
    return 1 / np.cosh(a)

In [ ]:
def phi_1(a):
    return -a + np.tanh(a)

def p_1(a):
    return np.cosh(a) * np.exp(-(a**2) / 2)

In [ ]:
def phi_2(a):
    return -a**3

def p_2(a):
    return np.exp(- (a**4) / 4)

In [ ]:
def phi_3(a):
    return -6*a / (a**2 + 5)

def p_3(a):
    return 1 / (a**2 + 5)**3

# # For fun: how does ReLU compares to the other activation fucntions?
# # Answer: It sucks! 
# # Theory of why it does not work

# def phi_4(a):
#     return np.maximum(a, 0)

# def p_4(a):
#     return np.maximum(a, 0)**2
    
# activation_functions = [phi_0, phi_1, phi_2, phi_3, phi_4]
# priors = [p_0, p_1, p_2, p_3, p_4]

# a = np.linspace(-5, 5, 1000)
# for prior in priors:
#     assert prior(a).shape == (1000, ), "Wrong output shape"

In [ ]:
activation_functions = [phi_0, phi_1, phi_2, phi_3]
priors = [p_0, p_1, p_2, p_3]

a = np.linspace(-5, 5, 1000)
for prior in priors:
    assert prior(a).shape == (1000, ), "Wrong output shape"


Plot the activation functions and the corresponding prior distributions, from $a = -5$ to $5$ (hint: use the lists defined in the cell above). Compare the shape of the priors to the histogram you plotted in the last question.

In [ ]:
### 1.3 Implicit priors (continued)

a = np.linspace(-5, 5, 1000)
f, ax = plt.subplots(2, len(priors))
f.set_figheight(10)
f.set_figwidth(20)

for i in range(len(priors)): 
    ax[0, i].plot(a, activation_functions[i](a), label = 'Activation')
    ax[1, i].plot(a, priors[i](a), label = 'Prior')
    ax[0, i].legend()
    ax[1, i].legend()
    ax[0, i].set_title(f'Activation phi_{i}')
    ax[1, i].set_title(f'Prior p_{i}')

***
**ANSWER:**

We see that all the activation funcitons correspond to priors that more or less look like the standard normal (Gaussian). These prior distributions correspond the most to the ***random*** source signal shape. Amongst the priors, the $p_0$ and $p_3$ look the most like a standrad Normal (due to their bell shape and the tail).
***

### 1.4 Whitening (15 points)
Some ICA algorithms can only learn from whitened data. Write a method `whiten(X)` that takes a $M \times T$ data matrix $\mathbf{X}$ (where $M$ is the dimensionality and $T$ the number of examples) and returns a whitened matrix. If you forgot what whitening is or how to compute it, various good sources are available online, such as http://courses.media.mit.edu/2010fall/mas622j/whiten.pdf. Your function should also center the data before whitening.

***
$cov[X] = \frac{1}{n-1} \sum_{i=1}^{n}\left(x_{i}-\overline{x}\right)^{2}$, where $\overline{x}=$ mean of x

$\mathbf{whitened(x)} = \boldsymbol{\Lambda}^{-1 / 2} \boldsymbol{\Phi}^{T} \mathbf{x}$

***

In [ ]:
### 1.4 Whitening
def whiten(X):
    
    # Center the data
    mean_X = np.mean(X, axis = 1, keepdims=True)
    X -= mean_X
    
    # Normalizes the covariance
    lambd, Phi = np.linalg.eig(np.cov(X))
    Lambd = np.diag(np.power(lambd, -1/2)) # put eigen values in the main diagonal
    A_w = Phi @ Lambd
    X_white = A_w.T @ X
    
    return X_white

In [ ]:
### Test your function
Xw = whiten(X)
assert Xw.shape == (num_sources, signal_length), "The shape of your mixed signals is incorrect"


### 1.5 Interpret results of whitening (10 points)
Make 3 figures, one for the sources, one for measurements and one for the whitened measurements. In each figure, make $5 \times 5$ subplots with scatter plots for each pair of signals. Each axis represents a signal and each time-instance is plotted as a dot in this space. You can use the `plt.scatter()` function. Describe what you see.

Now compute and visualize the covariance matrix of the sources, the measurements and the whitened measurements. You can visualize each covariance matrix using this code:
```python
# Dummy covariance matrix C;
C = np.eye(5)  
ax = imshow(C, cmap='gray', interpolation='nearest')
```

In [ ]:
### 1.5 Interpret results of whitening

mixtures = []
mix_whiten = []
scatter_colors = ['b' , 'r', 'g']

for s in signals:
    mixtures.append(s+'_mix')
    mix_whiten.append(s+'_whitened')


def plot_scatter(X, title, subtitles=[], count=0):
    
    dim = X.shape[0]
    f, axes = plt.subplots(dim, dim, figsize=(13, 13))
    f.suptitle(title + "\n" + "-"*80, y=1.04)
    
    for row in range(dim):
        for col in range(dim):
            ax = axes[row,col]
            ax.scatter(X[row, :],X[col, :], s= 0.6, c= scatter_colors[count])
            ax.set_xlabel(subtitles[row])
            ax.set_ylabel(subtitles[col])
    
    plt.tight_layout()
    return None
    
    
def plot_covariance(X, title, labels=[]):
    
    f, ax = plt.subplots(1, figsize=(4, 4))
    f.suptitle(title + "\n" + "-"*80, y=1.04)
    
    C = np.cov(X) 
    ax.set_xticks([0,1,2,3,4])
    ax.set_xticklabels(labels,rotation = 30, horizontalalignment="right")
    ax.set_yticks([0,1,2,3,4])
    ax.set_yticklabels(labels)
    ax = imshow(C, cmap='gray', interpolation='nearest')  

# Plotting Pairwise Scatterplots   
plot_scatter(S, 'Pairwise Source Signals Scatterplots', signals,0)
plot_scatter(X, 'Pairwise Measurements Scatterplots', mixtures,1)
plot_scatter(Xw, 'Pairwise Whitened Measurements Scatterplots', mix_whiten,2)

# Plotting Pairwise Covariance
plot_covariance(S, "Pairwise Source Signals Covariance", signals)
plot_covariance(X, "Pairwise Measurements Covariance", mixtures)
plot_covariance(Xw, "Pairwise Whitened Measurements Covariance", mix_whiten)

Are the signals independent after whitening?

***

**ANSWER:**

The Whitened Measurements are de-correlated, because the covariance is the identity matrix. However we cannot conclude they are independent. In other words, $\Sigma = I$ is a necessary condition, but it is not sufficient.

The scatterplots before show clear linear correlation between measurement pairs. This is also confirmed from the covariance matrix that has patches (and/or shades) of white all over, where white denotes $1$ (highest correlation) and black denotes $0$ (no correlation). 

After whitening, we can see the scatter plots are all blobs with no apparent rotation visible. Even the covariance matrix has white patches only on the diagonals (obvious high correlation with self) and no correaltion with others (black everywhere).

***

### 1.6 Covariance (5 points)
Explain what a covariant algorithm is. 

A Covariant algorithm should give the same result independent of the unit in which the quantities are measured. Notably, the Gradient Descent update $\Delta W = \eta \nabla_W L$ is not consistent, the left side of the equation is of dimensions $[W]$, the the right side is $[W]^{-1}$. Indeed, gradient descent is guaranteed to converge (under some conditions), but only assimptocially. 

A work around it is to multiply the right side of the equation by a matrix $M$. The Newton-Hapson update does exatly that, it locally aproximates the loss curve with a parabole and uses the hessian to perform the update. For quadratic and noise-free problems, one single update is sufficient to find the minimum. In this case the matrix $M$ encodes the curvature of the loss curve, other update rules can be designed using this curvature.

$M$ can be a curvature matrix, but it can also be a metric matrix. In this second case, if there is a metric that defines distances in the parameter space $W$, then $M$ can be obtained. We often use the quadratic metric of the euclidean space, in this case $M = I$. Which is quite convinient because that does not change anyhing on the update rule of gradient descent $\Delta W = \eta M \nabla_W L = \eta I \nabla_W L = \eta \nabla_W L$. 

More detailed information can also be found here: http://www.inference.org.uk/mackay/ica.pdf

### 1.7 Independent Component Analysis (25 points)
Implement the covariant ICA algorithm as described in MacKay. Write a function `ICA(X, activation_function, learning_rate)`, that returns the demixing matrix $\mathbf{W}$. The input `activation_function` should accept a function such as `lambda a: -tanh(a)`. Update the gradient in batch mode, averaging the gradients over the whole dataset for each update. Make it efficient, so use matrix operations instead of loops where possible (loops are slow in interpreted languages such as python and matlab, whereas matrix operations are internally computed using fast C code). Experiment with the learning rate and the initialization of $\mathbf{W}$. Your algorithm should be able to converge (i.e. `np.linalg.norm(grad) < 1e-5`) in within 10000 steps.

In [ ]:
### 1.7 Independent Component Analysis
def ICA(X, activation_function, learning_rate=0.25, print_result=True):
    
    # Initialize W as an orthogonal matrix
    # This reduces the number of updates that is necessary.
    W, _ = np.linalg.qr(random_nonsingular_matrix(Xw.shape[0]))
    
    max_iters = 10000
    
    ## IF LEARNING RATE IS TOO HIGH IT IS OVER WRITEN TO 0.25, AS INSTRUCTED.
    if learning_rate >= 1:
        learning_rate=0.25     
    
    for i in range(max_iters):
      
        # step 1
        a = W @ X
        
        # step 2
        z = activation_function(a)

        # step 3
        x_prime = W.T @ a
        
        # step 4
        grad = W + z @ x_prime.T / x_prime.shape[1] 
        
        # Update W
        W += learning_rate * grad
        
        # Check for the converging criterial
        if  np.linalg.norm(grad) < 1e-5:
            break
    
    if print_result:
        print(f'Number of iterations: {i +1}, grad: {np.linalg.norm(grad)} ')
    
    return W

lr = .25
W_est = ICA(Xw, phi_0, learning_rate=lr)  # Whitened measuement
W_est = ICA(X, phi_0, learning_rate=lr)   # raw measuerement
W_est = ICA(Xw, phi_1, learning_rate=lr)  
W_est = ICA(X, phi_1, learning_rate=lr)
W_est = ICA(Xw, phi_2, learning_rate=lr)  
W_est = ICA(X, phi_2, learning_rate=lr)   # Does not converge for anyting in the world!
# W_est = ICA(Xw, phi_3, learning_rate=lr)  
# W_est = ICA(X, phi_3, learning_rate=lr)

In [ ]:
# We will test your function so make sure it runs with only X and phi as input, and returns only W
# Also it should converge for all activation functions

W_estimates = [ICA(Xw, activation_function=phi, learning_rate=1.0) for phi in activation_functions]
assert all([W_est.shape == (num_sources, num_sources) for W_est in W_estimates])


### 1.8 Experiments  (5 points)
Run ICA on the provided signals using each activation function $\phi_0, \ldots, \phi_3$ (or reuse `W_estimates`). Use the found demixing matrix $\mathbf{W}$ to reconstruct the signals and plot the retreived signals for each choice of activation function.

In [ ]:
# 1.8 Experiments

def run_experiments(X_data, activation_functions, lr=0.25, title='', plot=True):

    for i, act_func in enumerate(activation_functions):
        W = ICA(X_data, act_func, learning_rate=lr, print_result=plot)
        if plot:
            print(f"Phi {i}: {title}")
            plot_signals(W @ Xw, f"Phi {i}: {title}")
    

lr =.25
# Experiments on white data
print("################# White Data #################")
run_experiments(Xw, activation_functions, lr=lr, title="De-mixed for whitened data")

# Experiments on raw data
print("################# Raw Data #################")
run_experiments(X, activation_functions, lr=lr, title="De-mixed for raw data")

In [ ]:
import time
def does_whitening_make_a_difference():
    # Does it make a difference (in terms of speed of convergence) 
    # if you whiten your data before running ICA?
    
#     # fact checking
#     lr=.25
    
#     # Time needed for white data
#     white_start = time.time()
#     run_experiments(Xw, activation_functions, lr=lr, title="De-mixed for whitened data", plot=False)
#     white_time = time.time() - white_start
    
#     # Time needed for raw data
#     raw_start = time.time()
#     run_experiments(Xw, activation_functions, lr=lr, title="De-mixed for raw data", plot=False)
#     raw_time = time.time() - raw_start
    
#     return white_time < raw_time

    return True

a = does_whitening_make_a_difference()
print(a)

In [ ]:
assert type(does_whitening_make_a_difference()) == bool

### 1.9 Audio demixing (10 points)
The 'cocktail party effect' refers to the ability humans have to attend to one speaker in a noisy room. We will now use ICA to solve a similar but somewhat idealized version of this problem. The code below loads 5 sound files and plots them.

Use a random non-singular mixing matrix to mix the 5 sound files. You can listen to the results in your browser using `play_signals`, or save them to disk if this does not work for you. Plot histograms of the mixed audio and use your ICA implementation to de-mix these and reproduce the original source signals. As in the previous exercise, try each of the activation functions.

Keep in mind that this problem is easier than the real cocktail party problem, because in real life there are often more sources than measurements (we have only two ears!), and the number of sources is unknown and variable. Also, mixing is not instantaneous in real life, because the sound from one source arrives at each ear at a different point in time. If you have time left, you can think of ways to deal with these issues.

In [ ]:
import scipy.io.wavfile
from IPython.display import Audio, display, Markdown

# Save mixtures to disk, so you can listen to them in your audio player
def save_wav(data, out_file, rate):
    scaled = np.int16(data / np.max(np.abs(data)) * 32767)
    scipy.io.wavfile.write(out_file, rate, scaled)

# Or play them in your browser
def play_signals(S, sample_rate, title="Signals"):
    display(Markdown(title))
    for signal in S:
        display(Audio(signal, rate=sample_rate))

In [ ]:
# Load audio sources
source_files = ['beet.wav', 'beet9.wav', 'beet92.wav', 'mike.wav', 'street.wav']
wav_data = []
sample_rate = None
for f in source_files:
    sr, data = scipy.io.wavfile.read(f, mmap=False)
    if sample_rate is None:
        sample_rate = sr
    else:
        assert(sample_rate == sr)
    wav_data.append(data[:190000])  # cut off the last part so that all signals have same length

# Create source and measurement data
S_audio = np.c_[wav_data]
plot_signals(S_audio)
play_signals(S_audio, sample_rate)

In [ ]:
### 1.9 Audio demixing

# Mixing the signals
A = random_nonsingular_matrix(d=S_audio.shape[0])
X = make_mixtures(S_audio, A)
# plot_signals(X, "Mixtures")
# play_signals(X, sample_rate, "Mixtures")

# Whitening the signals
Xw = whiten(X)
# play_signals(Xw, sample_rate, "Whitened mixtures")
# plot_signals(Xw, "Whitened mixtures")
Sw = whiten(S_audio.astype(float)) # useful to compare with the de-mixed data

# De-mixing signals
W_estimates_audio = []
lr = 0.1  # fine tunned for the given data.
for i, act_func in enumerate(activation_functions):
    W = ICA(Xw, act_func, learning_rate=lr)
    W_estimates_audio.append(W)
    play_signals(W @ Xw , sample_rate, f"Phi {i} de-mixed")
    plot_signals(W @ Xw, f"Phi {i} de-mixed")

Report your results. Using which activation functions ICA recovers the sources?

The sourcse were quite well reconvered by using the activation functions $\phi_0$ and $\phi_3$. It is even hard to distinguish between the recovered audio and the original one.

The other thwo activation functions did not perform that well. The recovered audios are still quite mixed, so much so it is hard to point the recovered audios to the original ones.

### 1.10 Excess Kurtosis (15 points)
The (excess) kurtosis is a measure of 'peakedness' of a distribution. It is defined as
$$
\verb+Kurt+[X] = \frac{\mu_4}{\sigma^4} - 3 = \frac{\operatorname{E}[(X-{\mu})^4]}{(\operatorname{E}[(X-{\mu})^2])^2} - 3
$$
Here, $\mu_4$ is known as the fourth moment about the mean, and $\sigma$ is the standard deviation.
The '-3' term is introduced so that a Gaussian random variable has 0 excess kurtosis.
We will now try to understand the performance of the various activation functions by considering the kurtosis of the corresponding priors, and comparing those to the empirical kurtosis of our data.

#### 1.10.1 (10 points)
First, compute analytically the kurtosis of the four priors that you derived from the activation functions before. You may find it helpful to use an online service such as [Wolfram Alpha](https://www.wolframalpha.com/) or [Integral Calculator](https://www.integral-calculator.com/) to (help you) evaluate the required integrals. Give your answer as both an exact expression as well as a numerical approximation (for example $\frac{\pi}{2} \approx 1.571$).

All priors are centered in zero, thus $\mu = 0$. Which makes the calculations easier.

First we have to calculate the normalizing constants of each prior:

$c_0 = \int_{-\infty}^{\infty} p_0(x) dx = \int_{-\infty}^{\infty} \frac{1}{cosh(x)} dx = \pi \approx 3.14$

$c_1 = \int_{-\infty}^{\infty} p_1(x) dx = \int_{-\infty}^{\infty} (cosh(x)exp(-\frac{x^2}{2})) dx = \sqrt{2 e \pi} \approx  4.13$

$c_2 = \int_{-\infty}^{\infty} p_2(x) dx = \int_{-\infty}^{\infty} exp(-\frac{x^4}{4}) dx = \frac{\Gamma(\frac{1}{4})}{\sqrt{2}} \approx 2.56$

$c_3 = \int_{-\infty}^{\infty} p_3(x) dx = \int_{-\infty}^{\infty} \frac{1}{(x^2 + 3)^3} dx = \frac{\pi}{24 \sqrt{3}} \approx 0.0075$

Now we calculate the $\sigma^4$ of each prior:

$\sigma_0^4 = \mathcal{E}[(x^2)]^2 =  \left( \int_{-\infty}^{\infty} p_0(x)x^2 dx \right)^2  = \left( \frac{1}{\pi} \int_{-\infty}^{\infty} \frac{x^2}{cosh(x)} dx \right)^2   = \frac{\pi^4}{16} \approx 6.08$

$\sigma_1^4 = \mathcal{E}[(x^2)]^2 =  \left( \int_{-\infty}^{\infty} p_1(x)x^2 dx \right)^2  = \left( \frac{1}{\sqrt{2 e \pi}} \int_{-\infty}^{\infty} (cosh(x)exp(-\frac{x^2}{2}))x^2 dx \right)^2   = 4$

$\sigma_2^4 = \mathcal{E}[(x^2)]^2 =  \left( \int_{-\infty}^{\infty} p_2(x)x^2 dx \right)^2  = \left( \frac{1}{\frac{\Gamma(\frac{1}{4})}{\sqrt{2}}} \int_{-\infty}^{\infty} exp(-\frac{x^4}{4})x^2 dx \right)^2 = 4 \frac{\Gamma(\frac{3}{4})^2}{\Gamma(\frac{1}{4})^2} \approx 0.45$

$\sigma_3^4 = \mathcal{E}[(x^2)]^2 =  \left( \int_{-\infty}^{\infty} p_3(x)x^2 dx \right)^2  = \left( \frac{1}{\frac{\pi}{24 \sqrt{3}}} \int_{-\infty}^{\infty} \frac{x^2}{(x^2 + 3)^3} dx \right)^2 = 1 $

The last piece of the puzzle $\mu4$ is calculated for each prior:

$\mu4_0 = \mathcal{E}[(x^4)] =  \left( \int_{-\infty}^{\infty} p_0(x)x^4 dx \right)^2  =\frac{1}{\pi}  \int_{-\infty}^{\infty} \frac{x^4}{cosh(x)} dx  = \frac{5\pi^4}{16}  \approx 30.44$

$\mu4_1 = \mathcal{E}[(x^4)] =  \left( \int_{-\infty}^{\infty} p_1(x)x^4 dx \right)^2  = \frac{1}{\sqrt{2 e \pi}} \int_{-\infty}^{\infty} (cosh(x)exp(-\frac{x^2}{2}))x^4 dx  = 10 $

$\mu4_2 = \mathcal{E}[(x^4)] =  \left( \int_{-\infty}^{\infty} p_2(x)x^4 dx \right)^2  = \frac{1}{\frac{\Gamma(\frac{1}{4})}{\sqrt{2}}} \int_{-\infty}^{\infty} exp(-\frac{x^4}{4})x^4 dx  = 1$

$\mu4_3 = \mathcal{E}[(x^4)] =  \left( \int_{-\infty}^{\infty} p_3(x)x^4 dx \right)^2  = \frac{1}{\frac{\pi}{24 \sqrt{3}}} \int_{-\infty}^{\infty}  \frac{x^4}{(x^2 + 3)^3} dx  = 9$

Finally, we can calculate the Kurtosis:

$K_0 = \frac{\mu4_0}{\sigma_0^4 } - 3 =  \frac{\frac{5\pi^4}{16}}{\frac{\pi^4}{16}} - 3 = 2 $

$K_1 = \frac{\mu4_1}{\sigma_1^4 } - 3 =  \frac{10}{4} - 3 = -0.5$

$K_2 = \frac{\mu4_2}{\sigma_2^4 } - 3 =  \frac{1}{4 \frac{\Gamma(\frac{3}{4})^2}{\Gamma(\frac{1}{4})^2}} - 3 \approx -0.81$

$K_3 = \frac{\mu4_3}{\sigma_3^4 } - 3 =  \frac{9}{1} - 3 =6$

In [ ]:
### Include your answer here (you can use math.gamma if needed)
def get_kurtosis():
    # Return a list with 4 numbers / expressions
    
    Ks = [2, -.5, np.power(math.gamma(1/4), 2)/(np.power(math.gamma(3/4), 2) * 4) - 3, 6]
    return Ks
    

In [ ]:
# Let's check
kurtosis = get_kurtosis()
print (kurtosis)
assert len(kurtosis) == 4


#### 1.10.2 (5 points)
Now use the `scipy.stats.kurtosis` function, with the `fisher` option set to `True`, to compute the empirical kurtosis of the dummy signals and the real audio signals.

Can you use this data to explain the performance of the various activation functions on the synthetic and real data?

In [ ]:
### 1.10.2 Excess Kurtosis

from scipy.stats import kurtosis

kurts_dummy = kurtosis(S, fisher=True, axis=1)
kurts_audio = kurtosis(S_audio, fisher=True, axis=1)
    
print(f"Dummy: {kurts_dummy}")
print(f"Audio: {kurts_audio}")